# Databricks CLI: Installation & Authentication

**Objective:**
In this notebook, we will learn about the Databricks Command Line Interface (CLI). The CLI allows you to interact with Databricks services (Workspaces and Accounts) using commands in your terminal. It is essentially a wrapper around the Databricks REST API.

**We will cover:**
1.  **Installation**: How to install Databricks CLI on Windows/Linux/Mac.
2.  **Authentication**:
    *   **U2M (User-to-Machine)**: Using browser-based login.
    *   **M2M (Machine-to-Machine)**: Using Service Principals (OAuth).
3.  **Configuration**: Understanding Profiles and the `.databrickscfg` file.
4.  **Hands-on Commands**: Managing Catalogs, Schemas, and Account-level objects.

---
**Prerequisites:**
*   A Databricks Workspace.
*   Admin access to the Databricks Account Console (for M2M setup).
*   A local terminal (Command Prompt, PowerShell, or Bash).

## 1. Installation

The installation method depends on your operating system.

### **Windows**
We can use `winget` (Windows Package Manager).
Open your **Command Prompt** or **PowerShell** and run:

```bash
winget search databricks
winget install Databricks.DatabricksCLI
```

### **Linux / macOS**
You can use curl or homebrew.

#### For macOS using Homebrew
brew tap databricks/tap
brew install databricks

#### For Linux using curl
curl -fsSL https://raw.githubusercontent.com/databricks/setup-cli/main/install.sh | sh


### **Verification**
To verify the installation, run the following command in your terminal. It should return the version (e.g., 0.2xx).

```bash
databricks -v
```

## 2. Authentication: User-to-Machine (U2M)

This method is ideal for local development where a human user is interacting with Databricks. It uses your browser to authenticate.

### **Step 1: Run the Auth Command**
In your terminal, run the following command. Replace `<your-workspace-url>` with your specific URL (e.g., `https://adb-12345.12.azuredatabricks.net`).

```bash
databricks auth login --host <your-workspace-url>
```

### **Step 2: Configure Profile Name**

The CLI will ask for a Profile Name. This acts as an alias for this connection.

    > Input: sk (or any name you prefer).

    > Action: A browser window will open. Log in with your Databricks credentials.

### **Step 3: Verify Authentication**
Once logged in, the terminal will confirm the profile is saved. You can list your profiles:

```bash
databricks auth profiles
```

## 3. Configuration: Where are profiles stored?

Databricks CLI stores connection profiles in a configuration file located in your user's home directory.

*   **File Path:** `~/.databrickscfg` (Linux/Mac) or `%USERPROFILE%\.databrickscfg` (Windows).
*   **Content Structure:** It stores the host URL and token/auth type.

To view the content on Windows:
```bash
notepad %USERPROFILE%\.databrickscfg
```

To view on Linux/Mac:
```bash
cat ~/.databrickscfg
```

## 4. Hands-on: Workspace Level Commands

Now that we are authenticated, let's run some commands against the workspace.

**Note:** Always use the `-p <profile-name>` flag if you have multiple profiles.

### **A. List Catalogs**
View all catalogs in Unity Catalog.

```bash
databricks catalogs list -p sk
```

### **B. Get Help**
To understand available commands for a specific service.

```bash
databricks catalogs --help
databricks schemas --help
```

### **C. Create a Schema**
Let's create a new schema named schema_cli inside the dev catalog.

**Command Syntax:**

databricks schemas create <schema_name> <catalog_name> -p <profile>

```bash
databricks schemas create schema_cli dev -p sk
```

**Go to your Databricks UI -> Catalog Explorer to verify the creation.**

## 5. Authentication: Machine-to-Machine (M2M)

This method is used for CI/CD pipelines (e.g., Azure DevOps, GitHub Actions) where no human interaction is possible. We use a **Service Principal**.

### **Step 1: Create Service Principal & Secret**
1.  Go to **Databricks Account Console** -> **User Management** -> **Service Principals**.
2.  Select (or create) a Service Principal (e.g., `self-azure-sp`).
3.  Go to **Credentials & Secrets** -> **Generate Secret**.
4.  **Important:** Copy the **Client ID** and the generated **Client Secret**.

### **Step 2: Configure `.databrickscfg` Manually**
Open your `.databrickscfg` file and append the following configuration block. This allows the CLI to act as the Service Principal.

```ini
[sp-workspace]
host          = <workspace-url>
client_id     = <service-principal-client-id>
client_secret = <service-principal-client-secret>

[sp-account]
host          = https://accounts.azuredatabricks.net
account_id    = <databricks-account-id>
client_id     = <service-principal-client-id>
client_secret = <service-principal-client-secret>
```
[sp-workspace]: Used for workspace-level commands (Notebooks, Jobs, Tables).

[sp-account]: Used for account-level commands (Metastores, User Management).

### **Step 3: Test M2M Authentication**
List Catalogs using Service Principal:

```bash
databricks catalogs list -p sp-workspace
```

## 6. Hands-on: Account Level Commands

With the Service Principal configured for the account level (`[sp-account]`), we can manage account resources.

### **A. List Metastores**
See all metastores attached to your account.

```bash
databricks account metastores list -p sp-account
```

### **B. List Account Groups**
See all user groups defined in the account console.

```bash
databricks account groups list -p sp-account
```

## 7. Summary & Next Steps

### **Summary**
1.  Installed Databricks CLI via `winget`.
2.  Authenticated using **U2M** (Browser) for local dev.
3.  Authenticated using **M2M** (Service Principal) for automation.
4.  Manipulated Unity Catalog objects (Catalogs/Schemas) via CLI.
5.  Accessed Account Console data via CLI.

### **Next Steps**
In the next video, we will explore **Databricks Asset Bundles (DABs)** using the CLI commands (`databricks bundle`) to manage and deploy code as a package.